In [4]:
from util.config import load_configuration

PROP_FILE = 'app.properties'
configs = load_configuration(PROP_FILE)

API_TYPE = configs.get("API_TYPE").data
API_VERSION = configs.get("API_VERSION").data
API_KEY = configs.get("API_KEY").data
API_BASE = configs.get("API_BASE").data
LLM_ENGINE_GPT35_16K=configs.get("LLM_ENGINE_GPT35_16K").data
ADA_MODEL=configs.get("ADA_MODEL").data

In [5]:
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=API_BASE,
    azure_deployment=ADA_MODEL,
     api_key=API_KEY,
    openai_api_version=API_VERSION,
)

llm = AzureChatOpenAI(
    azure_endpoint=API_BASE,
    api_key=API_KEY,
    azure_deployment=LLM_ENGINE_GPT35_16K,
    openai_api_version=API_VERSION,
)


In [11]:
pip install langchain_chroma

  Using cached langchain_chroma-0.1.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached chromadb-0.4.24-py3-none-any.whl.metadata (7.3 kB)
  Using cached build-1.2.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached opentelemetry_api-1.24.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.24.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.45b0-py3-none-any.whl.metadata (2.0 kB)
  Using cached opentelemetry_sdk-1.24.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached kubernetes-29.0.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached PyYAML-6.0.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached pyproject_hooks-1.1.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached importlib_metadata-7.0.0-py3-none-any.whl.metadata (4.9 kB)

In [1]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

ModuleNotFoundError: No module named 'langchain_chroma'

In [6]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

NameError: name 'WebBaseLoader' is not defined